### Fluid Structure Interaction (FSI) Setup
To illustrate FSI TO, we consider a **wall fluid-structure-interaction (FSI) optimisation benchmark**.  
The rectangular channel domain is denoted by $\Omega$ and is split into  

* **$\Omega_I$ (the fixed “wall”)** – density locked at $\rho=1$,  
* **$\Omega_D$ (the design region)** – density field $\rho(\mathbf x)\in[0,1]$ is optimised.

The coupled Navier–Stokes–Cauchy equations are solved using a Brinkman formulation so that fluid velocity $\mathbf u$, pressure $p$ and solid displacement $\mathbf d$ are obtained.

---

## Optimization problem  

$$
\begin{aligned}
\min_{\rho}\quad 
& J
    = 
      \mathbf u^{\mathsf T}\mathbf K(\rho)\mathbf u,
\\[6pt]
\text{s.t.}\quad 
& \mathbf R\!\bigl(\rho,\mathbf u,p,\mathbf d\bigr)=\mathbf 0, \\[4pt]
& g(\rho)=
  \frac{\displaystyle\sum_{e} \rho_e\,v_e}{V^\ast}-1 \;\le 0, \\[4pt]
& 0\le\rho_e\le1\qquad\forall\,e .
\end{aligned}
$$

---

### Symbols  

| Symbol | Description |
|--------|-------------|
| $\rho_e$ | element-wise material fraction variable |
| $v_e$   | element volume (area in 2-D) |
| $\mathbf u,\;p$ | fluid velocity and pressure |
| $\mathbf d$ | solid displacement |
| $K$ | Global structural stiffness|
| $\mathbf R$ | monolithic FSI residual (Brinkman-penalised Navier–Stokes + elasticity) |
| $V^\ast$ | maximum permitted solid volume |

---

### Numerical details  

* **Element-based material fraction parametrisation** with RAMP/Brinkman interpolation for fluid equations and SIMP for solid.  
* **MMA optimiser** updates $\rho$.  
* Gradients $\partial J/\partial\rho$ and $\partial g/\partial\rho$ are computed automatically in **JAX** via discrete adjoints.  
* Density filtering and Heaviside projection with $\beta$-continuation enforce a minimum length scale and yield near-binary designs.


**Boundary conditions**

* Inlet $\Gamma_W$: parabolic velocity profile with unit peak velocity.  
* Outlet $\Gamma_E$: fixed pressure $p=0$.  
* Upper and lower walls $\Gamma_N,\Gamma_S$: no-slip for the fluid, and $\mathbf d=\mathbf 0$ on bottom for the elasticity solve.  

The coupled Navier–Stokes–Cauchy equations are solved using a unified Brinkman formulation so that fluid velocity $\mathbf u$, pressure $p$ and solid displacement $\mathbf d$ are obtained simultaneously.

<img src="../../figures/fsi.png" alt="Channel" style="width: 500px;"/>


In [ ]:
import functools
import numpy as np
import jax.numpy as jnp
import jax
from jax.typing import ArrayLike
import matplotlib.pyplot as plt

import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.bc as _bc
import toflux.src.utils as _utils

import toflux.src.fe_struct as _fea_struct
import toflux.src.fe_fluid as _fea_fluid
import toflux.experiments.fsi.pressure_coupling as _fsi
import toflux.src.solver as _solv
import toflux.src.mma as _mma
import toflux.src.viz as _viz


_Ext = _utils.Extent
_FluidField = _fea_fluid.FluidField
_Direc = _utils.Direction
_cmap = _viz.fluid_cmap

jax.config.update("jax_enable_x64", True)


# Define Geometry and Mesh 

Below we build the **channel-flow** benchmark commonly used in wall FSI TO.

#### Geometry  
A simple channel-flow is described in a json and passed as the geometry. This geometry is passed to the mesher.

#### Discretisation  
* The domain $(\Omega)$ is partitioned into **bilinear quadrilateral (Q1) elements**.  
* All primary fields—velocity $(\mathbf u=(u_x,u_y))$, pressure (p\) **and** displacement (d\)—are interpolated with the same Q1 shape functions.

#### Numerical integration  
Element contributions (mass, convection, brinkman, viscous, stability terms and structural stiffness)  are evaluated with a $(2 \times 2)$ Gauss quadrature.

This mesh serves as the baseline for the finite-element Navier–Stokes solve and for any subsequent density-based topology-optimisation runs.

In [ ]:
geom = _geom.BrepGeometry("toflux/brep/channel_flow_wall.json")

fluid_mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=150,
  nely_desired=75,
  dofs_per_node=3,
  gauss_order=2,
)

struct_mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=150,
  nely_desired=75,
  dofs_per_node=2,
  gauss_order=2,
)

density = jnp.zeros(fluid_mesh.num_elems)
_viz.plot_grid_mesh(fluid_mesh, density)
_viz.plot_brep(geom)

# Design region visualization

In [ ]:
design_bbox = _mesher.BoundingBox(x=_Ext(min=0.5, max=1.5), y=_Ext(min=0.0, max=0.5))

elems_in_design_dom = _mesher.compute_point_indices_in_box(
  struct_mesh.elem_centers, design_bbox
)
elems_out_design_dom = np.logical_not(elems_in_design_dom)
density = jnp.zeros(struct_mesh.num_elems)
density = density.at[elems_out_design_dom].set(1.0)
_viz.plot_grid_mesh(struct_mesh, density)

In [ ]:
dens_filter = _utils.create_density_filter(
  struct_mesh.elem_centers,
  cutoff_distance=0.02 * struct_mesh.bounding_box.diag_length,
  filter_type=_utils.Filters.CIRCULAR,
)

## Material


For this fluid demonstration we keep the parameters dimensionless:

| Symbol | Meaning | Value |
|--------|---------|-------|
| $(\rho)$ | Mass density | **1** |
| $(\mu)$ | Dynamic viscosity | **1** |
| $(E)$ | Young's Modulus | **1e5** |
| $(\nu)$ | Poisson Ratio | **0.3** |


We also prepare the *inverse permeability* field used in the Brinkman term.  
A RAMP interpolation converts the design variable $(\gamma\in[0,1])$ into the damping coefficient $(\alpha)$. In the subsequent topology optimization the brinkman penalty value damps fluid flow in solid.

In [ ]:
struct_mat = _mat.StructuralMaterial(
  youngs_modulus=1.0e5,
  poissons_ratio=0.30,
)

deformation_model = _fea_struct.DeformationModel.SMALL

In [ ]:
fluid_mat = _mat.FluidMaterial(
  mass_density=1.0,
  dynamic_viscosity=1.0,
)

inv_permeability_ext = _utils.Extent(min=0.0, max=1.0e9)


# Boundray Conditions


| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet (left vertical edge)** | Dirichlet | $(u = parabolic)$, $(v = 0)$ |
| **Outlet (right vertical edge)** | Dirichlet v, p | \(p = 0\), \(v = 0\) |
| **Top wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |
| **Bottom wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |

**Characteristic velocity**

The inlet velocity is chosen to give a Reynolds number of 1:

$$
U_c \;=\; \frac{\mathrm{Re}\,\nu}{H},
\qquad
\text{Re} = 1,\;
\quad
\nu = \frac{\mu}{\rho},
\quad
H = \text{characteristic length}.
$$
For structural bottom edge is fixed.


In [ ]:
face_tol = fluid_mesh.elem_size[0] * 0.5

btm_faces = _bc.identify_faces(struct_mesh, edges=[geom.edges[0]], tol=face_tol)
n = len(btm_faces)
xv = (_Direc.X, jnp.zeros(n))
yv = (_Direc.Y, jnp.zeros(n))
btm_face_val = [xv, yv]

fixed_bc = _bc.DirichletBC(elem_faces=btm_faces, values=btm_face_val)
struct_bc = _bc.process_boundary_conditions([fixed_bc], struct_mesh)

In [ ]:
reynolds_num = 1.0
char_velocity = (
  reynolds_num * fluid_mat.kinematic_viscosity / (fluid_mesh.bounding_box.ly)
)
ramp_penal = 5.25e-6


# inlet condition
inlet_faces = _bc.identify_faces(fluid_mesh, edges=[geom.edges[3]], tol=face_tol)
n = len(inlet_faces)
x_nodes = jnp.linspace(-1.0, 1.0, n + 1)
u_nodes = char_velocity * (1.0 - x_nodes**2)
face_node_vals = jnp.stack([u_nodes[1:], u_nodes[:-1]], axis=1)
u_vel = (_FluidField.U_VEL, face_node_vals)
v_vel = (_FluidField.V_VEL, jnp.zeros_like(face_node_vals))
inlet_face_val = [u_vel, v_vel]

# outlet condition
outlet_faces = _bc.identify_faces(fluid_mesh, edges=[geom.edges[1]], tol=face_tol)
n = len(outlet_faces)
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
pres = (_FluidField.PRESSURE, jnp.zeros(n))
outlet_face_val = [v_vel, pres]

# wall condition
wall_faces = _bc.identify_faces(
  fluid_mesh, edges=[geom.edges[0], geom.edges[2]], tol=face_tol
)
n = len(wall_faces)
u_vel = (_FluidField.U_VEL, jnp.zeros(n))
v_vel = (_FluidField.V_VEL, jnp.zeros(n))
wall_face_val = [u_vel, v_vel]

inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="inlet")
outlet_bc = _bc.DirichletBC(
  elem_faces=outlet_faces, values=outlet_face_val, name="outlet"
)
wall_bc = _bc.DirichletBC(elem_faces=wall_faces, values=wall_face_val, name="wall")

fluid_bc_list = [inlet_bc, outlet_bc, wall_bc]
fluid_bc = _bc.process_boundary_conditions(fluid_bc_list, fluid_mesh)

_viz.plot_bc(fluid_bc_list, fluid_mesh)

## Solver

The incompressible Navier–Stokes–Brinkman system is **intrinsically non-linear** because of the convective term .
Our solver therefore employs a *damped/modified Newton–Raphson* loop. For more details see 'solver.py'. While for structural the physics is linear, the solver still performs a Newton–Raphson step; for a small-strain model this step converges in **one iteration**. 

In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.SCIPY_SPARSE,
    "scipy_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-6},
}

flow_solver = _fea_fluid.FluidSolver(
  mesh=fluid_mesh, bc=fluid_bc, material=fluid_mat, solver_settings=solver_settings
)

struct_solver = _fea_struct.FEA(
  mesh=struct_mesh,
  material=struct_mat,
  deformation_model=deformation_model,
  bc=struct_bc,
  solver_settings=solver_settings,
)

## Define the Wall

Here, we define the wall, which serves as the initial design and the base upon which material is added.

In [ ]:
init_design = np.zeros((struct_mesh.num_elems, 1)) + 1.0e-4

wall_bbox = _mesher.BoundingBox(
  x=_Ext(min=0.975, max=1.025),
  y=_Ext(min=0.0, max=0.5),
)

elems_in_wall = _mesher.compute_point_indices_in_box(
  struct_mesh.elem_centers, wall_bbox
)

init_design[elems_in_wall] = 1.0

_viz.plot_grid_mesh(struct_mesh, init_design.reshape(-1))

# Brinkman Penalty Visualization

We visualize the Brinkman penalty here. The initial RAMP penalty, determined by `brink_inter_factor`, allows fluid flow in the domain at the beginning of the optimization. As the optimization proceeds, the penalty becomes more stricter, restricting fluid flow in the solid regions.
Note: Lower inverse permeability/brinkman penalty allows fluid flow.

In [ ]:
def compute_brinkman_penalty(
  density: ArrayLike,
  ramp_penalty: float,
  inv_perm_ext: _utils.Extent,
) -> ArrayLike:
  """Compute the Brinkman penalty."""
  fluid_frac = 1.0 - density
  ramp_factor = (1.0 + ramp_penalty) / (fluid_frac + ramp_penalty)
  return inv_perm_ext.max - inv_perm_ext.range * fluid_frac * ramp_factor

In [ ]:
test_mat_frac = np.linspace(0.0, 1.0, 100)
brink_penalty_test = compute_brinkman_penalty(test_mat_frac, ramp_penal, inv_permeability_ext)
plt.plot(test_mat_frac, brink_penalty_test)
plt.xlabel("Material Fraction")
plt.ylabel("Brinkman Penalty")
plt.title("Brinkman Penalty vs Material Fraction")
plt.show()

# Objective

In [ ]:
@functools.partial(jax.jit, static_argnames=("struct_fe", "fluid_fe"))
def objective_function(
  density: jnp.ndarray,
  struct_fe: _fea_struct.FEA,
  fluid_fe: _fea_fluid.FluidSolver,
  penal: float,
  threshold_beta: float,
  ramp_penalty: float,
  disp_guess=None,
  press_vel_guess=None,
):
  def objective_wrapper(density):
    # filter
    filtered_density = dens_filter @ density
    density = _utils.threshold_filter(filtered_density, beta=threshold_beta)
    density = density.at[elems_out_design_dom, 0].set(0.0)
    density = density.at[elems_in_wall, 0].set(1.0)

    # solve fluid
    brinkman_penalty = compute_brinkman_penalty(
      density=density,
      ramp_penalty=ramp_penalty,
      inv_perm_ext=inv_permeability_ext,
    )
    press_vel = _solv.modified_newton_raphson_solve(
      fluid_fe, press_vel_guess, brinkman_penalty
    )
    # compute coupling force
    pressure = press_vel[0 : fluid_mesh.num_dofs : 3]
    coupling_force = _fsi.compute_pressure_force(pressure, struct_mesh, density)

    # solve structural
    penal_dens = density**penal + 1e-4
    lam, mu = struct_mat.lame_parameters
    lam, mu = lam * penal_dens, mu * penal_dens
    u = _solv.newton_raphson_solve(struct_fe, disp_guess, lam, mu, coupling_force)

    # compute compliance
    force = jnp.zeros((struct_mesh.num_dofs,))
    elem_forces = struct_fe.bc["elem_forces"] + coupling_force
    force = force.at[struct_mesh.elem_dof_mat].add(elem_forces)

    compliance = jnp.einsum("i, i -> ", u, force)
    return compliance, (u, press_vel, density)

  (compliance, (displacements, press_vel, density)), d_compliance = jax.value_and_grad(
    objective_wrapper, has_aux=True
  )(density)
  return compliance, d_compliance.reshape((-1, 1)), displacements, press_vel, density

# Constraint

We define a simple material constraint optimization problem where we restrict the maximum material/solid the design can assume. With the user defined `max_vol_frac` we compute the material constraint. Once again, the gradients are computed automatically via jax's `value_and_grad`.

In [ ]:
def constraint_function(density: ArrayLike, max_vol_frac: float, threshold_beta: float):
  def constraint_wrapper(density):
    filtered_density = dens_filter @ density
    density = _utils.threshold_filter(filtered_density, beta=threshold_beta)
    density = density.at[elems_out_design_dom, 0].set(0.0)
    density = density.at[elems_in_wall, 0].set(1.0)
    vol_frac = jnp.mean(density)
    jax.debug.print("Volume fraction: {vol_frac}", vol_frac=vol_frac)
    return (vol_frac / max_vol_frac) - 1.0

  vol_cons, d_vol_cons = jax.value_and_grad(constraint_wrapper)(density)
  return jnp.array([vol_cons]), d_vol_cons.T

## Optimize

Here, we define the optimization loop. We use the MMA optimizer. We begin the optimization with a uniform design corresponding to the maximum allowed material fraction. The optimization steps are illustrated in following flowchart:

In [ ]:
def optimize_design(
  init_design: ArrayLike,
  struct_fe: _fea_struct.FEA,
  fluid_fe: _fea_fluid.FluidSolver,
  max_vol_frac: float,
  max_iter: int,
  move_limit: float = 1e-2,
  kkt_tol: float = 1e-5,
  step_tol: float = 1e-5,
  plot_interval: int = 5,
):
  num_design_var = init_design.shape[0]
  num_cons = 1
  lower_bound = np.zeros((num_design_var, 1))
  upper_bound = np.ones((num_design_var, 1))
  mma_params = _mma.MMAParams(
    max_iter=max_iter,
    kkt_tol=kkt_tol,
    step_tol=step_tol,
    move_limit=move_limit,
    num_design_var=num_design_var,
    num_cons=num_cons,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
  )
  mma_state = _mma.init_mma(init_design, mma_params)

  disp = jnp.zeros((struct_fe.mesh.num_dofs,)) + 1.0e-8
  disp = disp.at[struct_fe.bc["fixed_dofs"]].set(struct_fe.bc["dirichlet_values"])

  press_vel = jnp.zeros((fluid_mesh.num_dofs,))
  press_vel = press_vel.at[fluid_bc["fixed_dofs"]].set(fluid_bc["dirichlet_values"])

  while not mma_state.is_converged:
    print("mma_state.epoch", mma_state.epoch)

    penal = min(8.0, 1.0 + 0.05 * mma_state.epoch)
    beta = min(32.0, 1.0 + mma_state.epoch * 0.15)

    objective, grad_obj, disp, press_vel, density = objective_function(
      mma_state.x, struct_fe, fluid_fe, penal, beta, ramp_penal, disp, press_vel
    )

    disp = jax.lax.stop_gradient(disp)
    disp = disp.at[struct_fe.bc["fixed_dofs"]].set(struct_fe.bc["dirichlet_values"])

    press_vel = jax.lax.stop_gradient(press_vel)
    press_vel = press_vel.at[fluid_bc["fixed_dofs"]].set(fluid_bc["dirichlet_values"])

    constr, grad_cons = constraint_function(mma_state.x, max_vol_frac, beta)

    # Zero out the gradients for the wall and fluid domain
    grad_cons = grad_cons.at[0, elems_out_design_dom].set(0.0)
    grad_obj = grad_obj.at[elems_out_design_dom, 0].set(0.0)

    grad_cons = grad_cons.at[0, elems_in_wall].set(0.0)
    grad_obj = grad_obj.at[elems_in_wall, 0].set(0.0)

    status = f"epoch {mma_state.epoch} J {objective:.2E} mc {constr[0]:.2F}"
    print(status)

    if mma_state.epoch % plot_interval == 0:
      fig, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(
        struct_fe.mesh,
        density.reshape(-1),
        ax=ax,
        val_range=(0.0, 1.0),
        colorbar=True,
        cmap=_cmap,
      )
      ax.set_aspect("equal")
      ax.spines[["top", "right", "left", "bottom"]].set_visible(False)
      ax.set_xticks([])
      ax.set_yticks([])
      plt.show()
      plt.pause(1e-6)

    mma_state = _mma.update_mma(
      mma_state, mma_params, objective, grad_obj, constr, grad_cons
    )

  return mma_state, disp, press_vel

In [ ]:
mma_state, disp, press_vel = optimize_design(
  init_design=init_design,
  struct_fe=struct_solver,
  fluid_fe=flow_solver,
  max_vol_frac=0.1,
  max_iter=150,
  move_limit=5.0e-2,
)